In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import pylab
import pickle
import numpy as np
import time
import itertools
from tqdm.notebook import tqdm
from multiprocessing import Pool
import matplotlib.cm as cm
import scipy
import community as community_louvain
import seaborn as sns
import pickle
import os
from collections import defaultdict
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from sklearn.metrics import accuracy_score,classification_report


/home/datahog/venvs/py_36/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('soc-redditHyperlinks-body.tsv',sep='\t')

In [28]:
nodes = df['SOURCE_SUBREDDIT'].unique().tolist()
nodes.extend(df['TARGET_SUBREDDIT'].unique().tolist())

In [29]:
G_base = nx.Graph()
G_base.add_nodes_from(nodes)
G_base.add_edges_from(list(zip(df['SOURCE_SUBREDDIT'].values.tolist(),
                          df['TARGET_SUBREDDIT'].values.tolist())))

In [30]:
comps = list(nx.connected_components(G_base))
connected_subgrpah_base = G_base.subgraph(comps[0])
nx.is_connected(connected_subgrpah)


True

In [19]:
from node2vec import Node2Vec


# Important Note!
Node2vec is really really slow when executing on large graphs, this took on my machine (20 cores,64gig Ram) roughly 14 hours. I do not suggest to rerun it since i'd guess it'll take days, temp_folder helps with ram consumption but then again might make things even more slow


In [34]:
node2vec = Node2Vec(connected_subgrpah_base, dimensions=64, walk_length=10, num_walks=200, workers=16)  # Use temp_folder for big graphs


In [35]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)  













Generating walks (CPU: 2):  15%|█▌        | 2/13 [41:14<3:46:47, 1237.04s/it]

Generating walks (CPU: 3):  15%|█▌        | 2/13 [41:15<3:46:53, 1237.63s/it]








Generating walks (CPU: 10):  17%|█▋        | 2/12 [41:15<3:26:15, 1237.58s/it]







Generating walks (CPU: 9):  17%|█▋        | 2/12 [41:15<3:26:18, 1237.87s/it]


Generating walks (CPU: 4):  15%|█▌        | 2/13 [41:16<3:47:00, 1238.27s/it]














Generating walks (CPU: 14):  17%|█▋        | 2/12 [41:15<3:26:18, 1237.87s/it]




Generating walks (CPU: 6):  15%|█▌        | 2/13 [41:16<3:47:00, 1238.21s/it]










Generating walks (CPU: 12):  17%|█▋        | 2/12 [41:16<3:26:20, 1238.02s/it]





Generating walks (CPU: 7):  15%|█▌        | 2/13 [41:16<3:47:01, 1238.28s/it]






Generating walks (CPU: 8):  15%|█▌        | 2/13 [41:16<3:47:01, 1238.35s/it]












Generating walks (CPU: 15):  17%|█▋        | 2/12 [41:17<3:26:28, 1238.80s/it]



Generating walks (CPU: 5):  15%|█▌        | 2/13 [41:1

Generating walks (CPU: 5):  85%|████████▍ | 11/13 [7:13:24<1:25:00, 2550.16s/it]












Generating walks (CPU: 15):  92%|█████████▏| 11/12 [7:13:24<42:30, 2550.16s/it]  


Generating walks (CPU: 4):  85%|████████▍ | 11/13 [7:13:26<1:25:01, 2550.70s/it]

Generating walks (CPU: 1):  85%|████████▍ | 11/13 [7:13:40<1:25:03, 2551.74s/it]













Generating walks (CPU: 2):  92%|█████████▏| 12/13 [7:56:17<42:42, 2562.27s/it]  








Generating walks (CPU: 10): 100%|██████████| 12/12 [7:56:23<00:00, 2562.64s/it]




Generating walks (CPU: 6):  92%|█████████▏| 12/13 [7:56:27<42:43, 2563.15s/it]  









Generating walks (CPU: 11): 100%|██████████| 12/12 [7:56:29<00:00, 2563.07s/it]










Generating walks (CPU: 12): 100%|██████████| 12/12 [7:56:33<00:00, 2563.44s/it]





Generating walks (CPU: 7):  92%|█████████▏| 12/13 [7:56:35<42:43, 2563.59s/it]  











Generating walks (CPU: 13): 100%|██████████| 12/12 [7:56:37<00:00, 2564.42s/it]














Generating walks (CPU: 

In [36]:
print('Done')

Done


In [37]:
model.wv.save_word2vec_format("embeddings_full")


In [38]:
model.save("embeddings_model_full")


In [44]:
model.wv.get_vector("leagueoflegends")

array([-2.5816503 ,  4.1973567 , -0.4141582 , -4.6376157 , -1.2704643 ,
        0.72188026,  3.3026035 ,  3.8525624 ,  0.08277021,  0.855891  ,
       -1.1154779 ,  3.7132666 ,  0.05496906,  2.0811074 , -3.8547535 ,
       -2.9224288 , -0.56850475, -0.43271646, -3.2355423 , -0.7292446 ,
        1.7024751 , -0.63628536,  0.32494637, -1.3318816 ,  3.4806697 ,
        5.4780884 , -2.0316787 ,  6.405602  , -3.7137918 ,  0.48166445,
       -3.0572314 ,  0.8913412 , -1.0101666 , -4.2819195 , -2.1988974 ,
       -1.6222624 ,  2.2257314 ,  2.691602  , -2.1002238 , -1.8447227 ,
        4.395832  , -1.3371931 , -1.7684337 ,  2.3365197 ,  5.4313946 ,
       -3.0633633 ,  0.64677817,  2.7057843 ,  0.36577612, -0.7225612 ,
       -1.5694673 , -4.6154346 ,  1.0422554 ,  2.6388407 , -1.30951   ,
        2.200328  , -0.53780025, -0.94485724, -3.0691578 ,  3.8823278 ,
        4.6423044 , -4.706831  , -1.9370724 ,  6.7495832 ], dtype=float32)

In [45]:
model.wv.most_similar('leagueoflegends')

[('0bservat0ry', 0.6100233793258667),
 ('competeleague', 0.5878720283508301),
 ('teamfusiongaming', 0.5816200971603394),
 ('malphitemains', 0.5812757015228271),
 ('amumumains', 0.5808323621749878),
 ('vaynemains', 0.573606550693512),
 ('redditbroscjsesh', 0.5730637311935425),
 ('lolwallpaper', 0.5715264081954956),
 ('leagueoflegendsfr', 0.570448100566864),
 ('inven', 0.5701046586036682)]

In [46]:
model.wv.most_similar("reactiongifs")

[('musicrewritten', 0.6684017181396484),
 ('fffffffuuuuuuuuuuuu', 0.6606501936912537),
 ('lititz', 0.6550098061561584),
 ('veganpogrom', 0.654785692691803),
 ('hohsi', 0.6375021934509277),
 ('perfecttiming', 0.6296089887619019),
 ('thathappened', 0.6263118386268616),
 ('goodfaketexts', 0.6231029033660889),
 ('wtfstockphotos', 0.6136131286621094),
 ('nononono', 0.6125208139419556)]

In [49]:
model.wv.most_similar("contagiouslaughter")

[('twitchfunnymoments', 0.7090020179748535),
 ('gethighandwrite', 0.6962473392486572),
 ('wastedgifs', 0.5509545803070068),
 ('prematurecelebration', 0.5470633506774902),
 ('bestoftldr', 0.5315432548522949),
 ('3amjokes', 0.5312332510948181),
 ('nobackspace', 0.5123492479324341),
 ('fuck2016', 0.49437016248703003),
 ('fffffffuuuuuuuuuuuu', 0.49308907985687256),
 ('fatpeoplehate66', 0.4783671796321869)]

In [50]:
model.wv.most_similar("nononono")

[('thathappened', 0.6725264191627502),
 ('gatekeeping', 0.6566230058670044),
 ('whatcouldgowrong', 0.6563301086425781),
 ('fffffffuuuuuuuuuuuu', 0.6526663899421692),
 ('veganpogrom', 0.6456226110458374),
 ('musicrewritten', 0.6400573253631592),
 ('goodfaketexts', 0.634431004524231),
 ('expectationvsreality', 0.6318814754486084),
 ('holdmybeer', 0.6315089464187622),
 ('youtubehaiku', 0.6240314245223999)]

In [54]:
model.wv.most_similar("2meirl4meirl")

[('2meirl42meirl4meirl', 0.7165687084197998),
 ('ultrameirl', 0.6844120025634766),
 ('bestoftldr', 0.6784727573394775),
 ('kingofthehill', 0.6676276922225952),
 ('coloringcorruptions', 0.6516022682189941),
 ('liamariejohnson', 0.6442063450813293),
 ('ignorantimgur', 0.6436528563499451),
 ('meirl', 0.6311373710632324),
 ('thesquadonpoint', 0.6272146701812744),
 ('aliensamongus', 0.6243982911109924)]

In [56]:
data = list()
for nodename in tqdm(list(connected_subgrpah_base.nodes)):
    data.append(model.wv.get_vector(nodename))

In [79]:
data_df = pd.DataFrame(data)
data_df['subreddit'] = list(connected_subgrpah_base.nodes)

In [80]:
data_df.to_csv('embeddings.csv',index=False)

In [1]:
from gensim.models import Word2Vec


In [2]:
model = Word2Vec.load("embeddings_model_full")

In [4]:
model.wv.most_similar("nononono")

[('thathappened', 0.6725264191627502),
 ('gatekeeping', 0.6566230058670044),
 ('whatcouldgowrong', 0.6563301086425781),
 ('fffffffuuuuuuuuuuuu', 0.6526663899421692),
 ('veganpogrom', 0.6456226110458374),
 ('musicrewritten', 0.6400573253631592),
 ('goodfaketexts', 0.634431004524231),
 ('expectationvsreality', 0.6318814754486084),
 ('holdmybeer', 0.6315089464187622),
 ('youtubehaiku', 0.6240314245223999)]